## Model Evaluation 

In [5]:
# import dependencies 
import math 

In [6]:
# Define input variables  
groups = [[1, 2, 3], [1], [0, 2]] 
votes = [5, 5, 5, 5]

In [7]:
def group_memberships(groups, votes):
    """
    Define group memberships for each participant.
    :param: groups (list of lists): a list denotes the group and contains its members. 
    :param: votes (list): number of participant's votes for a given project proposal.
    :returns (list of lists): retunrs a list of group membersips for each participant.  
    """
    # let each entry in votes define a unique identifyer for an agent 
    group_memberships = [[] for _ in range(len(votes))]

    # add group memberships of agent i as a list to each entry identifying an agent 
    for i in range(len(groups)):
        for j in groups[i]:
            group_memberships[j].append(i)
        
    return group_memberships

In [8]:
group_memberships = group_memberships(groups, votes)
print(group_memberships)

[[2], [0, 1], [0, 2], [0]]


In [9]:
def common_group(agent_i, agent_j):
    """
    Define an identifyer indicating whether two participants share any common group. 
    :param: agent_i denotes a participant not equal to a participant called agent_j. 
    :param: agent_j denotes a participant not equal to a participant called agent_i.
    :returns (bool): returns true if two participants share a common group and false otherwise.  
    """
    common_group = any(group in group_memberships[agent_j] for group in group_memberships[agent_i])

    return common_group

In [10]:
def K(agent_i, group):
    """
    Define the weighting function that attenuates the votes of agent i given different group memberships.  
    :param: agent_i denotes a participant not equal to a participant called agent_j.
    :param: (integer): group denotes the group number.
    :returns: attenuated number of votes for a given project. 
    """
    if agent_i in group or any(common_group(agent_i, j) for j in group):
        return math.sqrt(votes[agent_i])
    else:
        return votes[agent_i]


In [29]:
def first_term(groups, votes, group_memberships):
    """
    Calculate the first term of the formula for "connection-oriented cluster match" by individual and in aggregate.
    :param: groups (list of lists): a list denotes the group and contains its members.
    :param: votes (list): number of participant's votes for a given project proposal.
    :param: group_membership: defines group memberships for each participant.
    """ 
    weighted_votes_groups = {}
    for group_num, group in enumerate(groups):
        
        weighted_votes_individual = {}
        for agent_i in group:

            votes_i = votes[agent_i]
            num_groups_i = len(group_memberships[agent_i])
            vote_weight_i = votes_i/num_groups_i
            
            agent_key = f"agent_{agent_i}"
            weighted_votes_individual[agent_key] = {
                    'votes': votes_i,
                    'num_groups': num_groups_i,
                    'vote_weight': vote_weight_i
                }

        group_key = f"group_{group_num}"
        weighted_votes_groups[group_key] = weighted_votes_individual 
    
    aggregate_votes_weight = sum(agent_data["vote_weight"] for group_data in weighted_votes_groups.values() for agent_data in group_data.values())
    
    return weighted_votes_groups, aggregate_votes_weight

In [30]:
result_groups, result_aggregate_weight = first_term(groups, votes, group_memberships)
print(result_groups)
print(result_aggregate_weight)

{'group_0': {'agent_1': {'votes': 5, 'num_groups': 2, 'vote_weight': 2.5}, 'agent_2': {'votes': 5, 'num_groups': 2, 'vote_weight': 2.5}, 'agent_3': {'votes': 5, 'num_groups': 1, 'vote_weight': 5.0}}, 'group_1': {'agent_1': {'votes': 5, 'num_groups': 2, 'vote_weight': 2.5}}, 'group_2': {'agent_0': {'votes': 5, 'num_groups': 1, 'vote_weight': 5.0}, 'agent_2': {'votes': 5, 'num_groups': 2, 'vote_weight': 2.5}}}
20.0


In [37]:
def interaction_terms(groups, group_memberships):
    """
    Calculate the interaction terms of the formula for "connection-oriented cluster match" by individual and in aggregate.
    :param: groups (list of lists): a list denotes the group and contains its members.
    :param: votes (list): number of participant's votes for a given project proposal.
    :param: group_membership: defines group memberships for each participant.
    """ 
    interaction_terms_individual = {}
    for group_num, group in enumerate(groups):
        interaction_terms_group = {}

        for other_group_num, other_group in enumerate(groups):
            if group_num == other_group_num: # index comparision allows to compare groups that have the same composition of participants 
                continue
            
            result_agent_i = {} 
            for agent_i in group: 
                vote_attenuation_i = K(agent_i, other_group) 
                num_groups_i = len(group_memberships[agent_i])
                vote_weight_i = vote_attenuation_i/num_groups_i

                agent_i_key = f"agent_i_{agent_i}"
                result_agent_i[agent_i_key] = {
                    'vote_attenuation_i': vote_attenuation_i,
                    'num_groups_i': num_groups_i,
                    'vote_weight_i': vote_weight_i
                }
                
            result_agent_j = {} 
            for agent_j in other_group:
                vote_attenuation_j = K(agent_j, group)
                num_groups_j = len(group_memberships[agent_j])
                vote_weight_j = vote_attenuation_j/num_groups_j

                agent_j_key = f"agent_j_{agent_j}"
                result_agent_j[agent_j_key] = {
                    'vote_attenuation_j': vote_attenuation_j,
                    'num_groups_j': num_groups_j,
                    'vote_weight_j': vote_weight_j
                }

            # Caluclation of individual internaction terms 
            sum_vote_weight_i = sum(value['vote_weight_i'] for value in result_agent_i.values())
            sum_vote_weight_j = sum(value['vote_weight_j'] for value in result_agent_j.values()) 
            sqrt_sum_vote_weight_i = math.sqrt(sum_vote_weight_i)
            sqrt_sum_vote_weight_j = math.sqrt(sum_vote_weight_j)

            #interaction_term1 = math.sqrt(sum(K(agent_i, other_group) / len(group_memberships[agent_i]) for agent_i in group))
            #interaction_term2 = math.sqrt(sum(K(agent_j, group) / len(group_memberships[agent_j]) for agent_j in other_group))

            other_group_key = f"other_group_{other_group_num}"
            interaction_terms_group[other_group_key] = {
                'components_int_term1': result_agent_i,
                'components_int_term2': result_agent_j,
                'sum_vote_weight_i': sum_vote_weight_i,
                'sqrt_sum_vote_weight_i': sqrt_sum_vote_weight_i,
                'sum_vote_weight_j': sum_vote_weight_j,
                'sqrt_sum_vote_weight_j': sqrt_sum_vote_weight_j,
                'multiplied_interaction': sqrt_sum_vote_weight_i*sqrt_sum_vote_weight_j 
            }

        group_key = f"group_{group_num}"
        interaction_terms_individual[group_key] = interaction_terms_group
    
    aggregated_interaction_terms = sum(value['multiplied_interaction'] for group_data in interaction_terms_individual.values() for value in group_data.values())
    
    return interaction_terms_individual, aggregated_interaction_terms

In [38]:
result_individual, result_aggregated = interaction_terms(groups, group_memberships)
print(result_individual)
print(result_aggregated)

{'group_0': {'other_group_1': {'components_int_term1': {'agent_i_1': {'vote_attenuation_i': 2.23606797749979, 'num_groups_i': 2, 'vote_weight_i': 1.118033988749895}, 'agent_i_2': {'vote_attenuation_i': 2.23606797749979, 'num_groups_i': 2, 'vote_weight_i': 1.118033988749895}, 'agent_i_3': {'vote_attenuation_i': 2.23606797749979, 'num_groups_i': 1, 'vote_weight_i': 2.23606797749979}}, 'components_int_term2': {'agent_j_1': {'vote_attenuation_j': 2.23606797749979, 'num_groups_j': 2, 'vote_weight_j': 1.118033988749895}}, 'sum_vote_weight_i': 4.47213595499958, 'sqrt_sum_vote_weight_i': 2.114742526881128, 'sum_vote_weight_j': 1.118033988749895, 'sqrt_sum_vote_weight_j': 1.057371263440564, 'multiplied_interaction': 2.2360679774997894}, 'other_group_2': {'components_int_term1': {'agent_i_1': {'vote_attenuation_i': 2.23606797749979, 'num_groups_i': 2, 'vote_weight_i': 1.118033988749895}, 'agent_i_2': {'vote_attenuation_i': 2.23606797749979, 'num_groups_i': 2, 'vote_weight_i': 1.118033988749895},

In [3]:
# print(math.sqrt(20+17.44884635887628))

In [86]:
def connection_oriented_cluster_match(groups, group_memberships):
    """
    This function calculates the connection-oriented cluster match of votes 
    for each agent and returns the attenuated sum of votes for a given project proposal.
    param: groups (list of lists): a list denotes the group and contains its members.
    param: votes (list): number of participant's votes for a given project proposal.
    param: group_membership: defines group memberships for each participant.
    """ 
    weighted_votes = 0

    # First term of the connection oriented cluster match formula 
    weighted_votes += first_term[1]

    # interaction terms of the connection oriented cluster match formula 
    for group in groups:
        for other_group in groups:
            if group == other_group:
                continue

            interaction_term1 = math.sqrt(sum(K(agent_i, other_group) / len(group_memberships[agent_i]) for agent_i in group))
            interaction_term2 = math.sqrt(sum(K(agent_j, group) / len(group_memberships[agent_j]) for agent_j in other_group))

            weighted_votes += interaction_term1 * interaction_term2
    
    # Transformation from quadratic finance to quadratic voting 
    sqrt_weighted_votes = math.sqrt(weighted_votes)

    return sqrt_weighted_votes

In [4]:
# result = connection_oriented_cluster_match(groups, group_memberships)
# print(result)